# matplotlib_slices_plotly_3d

Purpose: to be able to see slices of the subvolume on all three axis + Plotly 3D rendering in one place.

In [ ]:
! pip install import-ipynb

In [ ]:
#import MiscellaneousFunctions.ipynb as misc
import import_ipynb
import miscellaneous_functions as misc
import numpy as np

## Load numpy 3D array

In [12]:
subvolume = misc.numpy_binary_to_array("/content/test_medium.npy")
# subvolume = misc.stack_to_array(stack_path)

In [13]:
title = "test volume"
outfile = None  # e.g., 'test.png'
x_slices, y_slices, z_slices = np.shape(subvolume)

## 2D slices along x, y, z axes

In [8]:
import matplotlib.pyplot as plt
import math

In [10]:
imgs_in_row = 6
cmap_choice="jet"

In [ ]:
# Display x slices
num_rows =  math.ceil(x_slices/imgs_in_row)
figsize = (imgs_in_row*y_slices/10, num_rows*z_slices/6)
f, ax_arr = plt.subplots(num_rows, imgs_in_row, figsize=figsize)

for j, row in enumerate(ax_arr):
    for i, ax in enumerate(row):
      if j*imgs_in_row+i < x_slices:
        ax.imshow(subvolume[j*imgs_in_row+i, :, :],cmap=cmap_choice)
        ax.set_title(f'x-slice {j*imgs_in_row+i}')
  
f.suptitle(title, fontsize=16)
if outfile:
    plt.savefig(f'x_{outfile}')
plt.show()

In [ ]:
# Display y slices
num_rows =  math.ceil(y_slices/imgs_in_row)
figsize = (imgs_in_row*z_slices/10, num_rows*x_slices/6)
f, ax_arr = plt.subplots(num_rows, imgs_in_row, figsize=figsize)

for j, row in enumerate(ax_arr):
    for i, ax in enumerate(row):
      if j*imgs_in_row+i < y_slices:
        ax.imshow(subvolume[:,j*imgs_in_row+i, :],cmap=cmap_choice)
        ax.set_title(f'y-slice {j*imgs_in_row+i}')
  
f.suptitle(title, fontsize=16)
if outfile:
    plt.savefig(f'y_{outfile}')
plt.show()

In [ ]:
# Display z slices
num_rows =  math.ceil(z_slices/imgs_in_row)
figsize = (imgs_in_row*y_slices/10, num_rows*x_slices/6)
f, ax_arr = plt.subplots(num_rows, imgs_in_row, figsize=figsize)
for j, row in enumerate(ax_arr):
    for i, ax in enumerate(row):
      if j*imgs_in_row+i < z_slices:
        ax.imshow(subvolume[:,:, j*imgs_in_row+i], cmap=cmap_choice)
        ax.set_title(f'z-slice {j*imgs_in_row+i}')
  
f.suptitle(title, fontsize=16)
if outfile:
    plt.savefig(f'z_{outfile}')

plt.show()

## 3D rendering using Plotly

In [1]:
! pip install --upgrade "kaleido==0.1.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import plotly
plotly.__version__

'5.5.0'

In [3]:
import kaleido
kaleido.__version__

'0.1.0'

In [ ]:
import plotly.graph_objects as go

#voxel_size_um = 12.0
voxel_size_um = 1

#### 
### There seems to be a strange bug or something that prevents this 3D size from plotting correctly
### When it is np.mgrid[0:24, 0:64, 0:64], it does not result in a cube
### but when it is [0:12, ....] it does
### 

X, Y, Z = np.mgrid[0:x_slices, 0:y_slices, 0:z_slices]
#X, Y, Z = np.mgrid[0:16, 0:64, 0:64]

vol = go.Volume(
      name="volume rendering",
      x = X.flatten(),
      y = Y.flatten(),
      z = Z.flatten(),
      value = subvolume.flatten(),
      opacity = 0.3,
      opacityscale = 0.3,
      surface_count = 10,
      colorscale = 'jet',
      isomax = 40000,
      #isomin = 20000,
      #slices_z = dict(show=True, locations=[10]),
    )
fig = go.Figure(data=vol)


def generate_ticks(axis, interval, size_um=None):
  vals=[]
  ticks =[]

  if not size_um:
    size_um=1
  for i in range(0, axis, interval):
    vals.append(i)
    ticks.append(i*size_um)
  return (vals, [str(tick) for tick in ticks])

x_vals, x_ticks = generate_ticks(x_slices, 8)
y_vals, y_ticks = generate_ticks(y_slices, 8)
z_vals, z_ticks = generate_ticks(z_slices, 8)

fig.update_layout(title="test",
                  scene = dict(
                    xaxis = dict(
                        ticktext=x_ticks,
                        tickvals=x_vals),
                    yaxis = dict(
                        ticktext=y_ticks,
                        tickvals=y_vals),
                    zaxis = dict(
                        ticktext=z_ticks,
                        tickvals=z_vals)),
                 scene_aspectmode='data')

if outfile:
    fig.to_image(format="png", engine="kaleido")
    fig.write_image(f'plotly_{outfile}')

fig.show()